In [1]:
import os


In [2]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'f:\\Files\\DS&ML\\Flight-Fare-Price-Prediction'

In [5]:
from dataclasses import dataclass
from pathlib import Path

In [6]:
@dataclass
class ModelEvaluationConfig:
    root_dir:Path
    model_path:Path
    test_data_path:Path
    all_params:dict
    metric_file_path:Path
    preprocessor_path: Path
    target_column:str

In [7]:
from mlproject.utils.common import *
from mlproject.constants import *

[2025-02-21 23:34:57,994 : INFO : __init__ : Logger has been set up successfully!]


In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath=CONFIG_FILE_PATH,
        params_filepath=PARAMS_FILE_PATH,
        schema_filepath=SCHEMA_FILE_PATH,
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config["artifacts_root"]])


    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config["model_evaluation"]
        params = self.params["XGBRegressor"]
        schema = self.schema["TARGET_COLUMN"]

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path=config.model_path,
            all_params=params,
            metric_file_path=config.metric_file_path,
            preprocessor_path=config.preprocessor_path,
            target_column=schema.name,
        )
        return model_evaluation_config

In [9]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pandas as pd
import joblib
import json
import mlflow
import mlflow.sklearn
import dagshub

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

        # Initialize MLflow tracking
        os.environ['MLFLOW_TRACKING_USERNAME'] = "JavithNaseem-J"
        os.environ['MLFLOW_TRACKING_PASSWORD'] = "f86a04d73e27422c7a114f3f76b93fbc81a114fe"
        
        dagshub.init(repo_owner="JavithNaseem-J", repo_name="Laptop-Price-Prediction")
        mlflow.set_tracking_uri("https://dagshub.com/JavithNaseem-J/Laptop-Price-Prediction.mlflow")
        mlflow.set_experiment("Laptop Price Prediction")

    def evaluate(self):
        try:
            # Validate file paths
            if not os.path.exists(self.config.test_data_path):
                raise FileNotFoundError(f"Test data file not found at {self.config.test_data_path}")
            if not os.path.exists(self.config.preprocessor_path):
                raise FileNotFoundError(f"Preprocessor file not found at {self.config.preprocessor_path}")
            if not os.path.exists(self.config.model_path):
                raise FileNotFoundError(f"Model file not found at {self.config.model_path}")

            with mlflow.start_run():
                # Set tags for the run
                mlflow.set_tag("model_type", "LGBMRegressor")
                mlflow.set_tag("evaluation_stage", "testing")

                # Load preprocessor and model
                logger.info("Loading preprocessor and model...")
                preprocessor = joblib.load(self.config.preprocessor_path)
                model = joblib.load(self.config.model_path)

                # Log model parameters
                mlflow.log_params(self.config.all_params)

                # Load test data
                logger.info(f"Loading test data from {self.config.test_data_path}...")
                test_data = pd.read_csv(self.config.test_data_path)

                # Extract target column
                if self.config.target_column not in test_data.columns:
                    raise KeyError(f"Target column '{self.config.target_column}' not found in test data")

                test_y = test_data[self.config.target_column]
                test_x = test_data.drop(columns=[self.config.target_column])

                logger.info(f"Test data shape: X={test_x.shape}, y={test_y.shape}")

                # Preprocess test features
                logger.info("Preprocessing test features...")
                test_x_transformed = preprocessor.transform(test_x)

                # Make predictions
                logger.info("Making predictions on the test data...")
                predictions = model.predict(test_x_transformed)

                # Calculate and log metrics
                logger.info("Evaluating model performance...")
                mse = mean_squared_error(test_y, predictions)
                mae = mean_absolute_error(test_y, predictions)
                r2 = r2_score(test_y, predictions)
                adjusted_r2 = 1 - (1 - r2) * ((test_x.shape[0] - 1) / (test_x.shape[0] - test_x.shape[1] - 1))

                logger.info(f"Model Evaluation Metrics:\nMSE: {mse}\nMAE: {mae}\nR2: {r2}\nAdjusted R2: {adjusted_r2}")

                # Save the evaluation metrics
                metrics_path = os.path.join(self.config.root_dir, "metrics.json")
                metrics = {
                    "mean_squared_error": mse,
                    "mean_absolute_error": mae,
                    "r2_score": r2,
                    "adjusted_r2_score": adjusted_r2
                }
                # Log metrics to MLflow
                mlflow.log_metrics(metrics)

                # Log model with signature
                signature = mlflow.models.infer_signature(
                    test_x_transformed, predictions
                )
                mlflow.sklearn.log_model(
                    model,
                    "wine_quality_model",
                    signature=signature,
                    registered_model_name="Flight Fare Prediction"
                )

                logger.info(f"Model Evaluation Metrics:\naccuracy: {metrics['accuracy']}\n"
                          f"precision: {metrics['precision']}\nrecall: {metrics['recall']}\n"
                          f"f1: {metrics['f1']}")

                # Save metrics locally
                metrics_path = os.path.join(self.config.root_dir, "metrics.json")
                with open(metrics_path, "w") as f:
                    json.dump(metrics, f, indent=4)
                logger.info(f"Evaluation metrics saved at {metrics_path}")

                return metrics

        except Exception as e:
            logger.error(f"Error in model evaluation: {str(e)}")

In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(config=model_evaluation_config)
    metrics = model_evaluation.evaluate()

except FileNotFoundError as e:
    logger.error(f"File not found: {e}")
except KeyError as e:
    logger.error(f"Missing key in configuration or data: {e}")
except Exception as e:
    logger.error(f"Unexpected error: {e}")

[2025-02-21 23:35:04,147 : INFO : common : yaml file: config\config.yaml loaded successfully]
[2025-02-21 23:35:04,153 : INFO : common : yaml file: params.yaml loaded successfully]
[2025-02-21 23:35:04,156 : INFO : common : yaml file: schema.yaml loaded successfully]
[2025-02-21 23:35:04,158 : INFO : common : created directory at: artifacts]
[2025-02-21 23:35:04,160 : INFO : common : created directory at: artifacts/model_evaluation]


Accessing as JavithNaseem-J

Initialized MLflow to track repo "JavithNaseem-J/Laptop-Price-Prediction"

Repository JavithNaseem-J/Laptop-Price-Prediction initialized!

[2025-02-21 23:35:07,527 : INFO : 2610076619 : Loading preprocessor and model...]
[2025-02-21 23:35:09,036 : INFO : 2610076619 : Loading test data from artifacts/data_transformation/test.csv...]
[2025-02-21 23:35:09,225 : INFO : 2610076619 : Test data shape: X=(75039, 8), y=(75039,)]
[2025-02-21 23:35:09,226 : INFO : 2610076619 : Preprocessing test features...]
[2025-02-21 23:35:09,397 : INFO : 2610076619 : Making predictions on the test data...]
[2025-02-21 23:35:09,757 : INFO : 2610076619 : Evaluating model performance...]
[2025-02-21 23:35:09,773 : INFO : 2610076619 : Model Evaluation Metrics:
MSE: 14129786.0
MAE: 1995.400634765625
R2: 0.9725964069366455
Adjusted R2: 0.9725934850554713]


Registered model 'Flight Fare Prediction' already exists. Creating a new version of this model...
2025/02/21 23:35:28 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Flight Fare Prediction, version 7
Created version '7' of model 'Flight Fare Prediction'.


🏃 View run debonair-seal-555 at: https://dagshub.com/JavithNaseem-J/Laptop-Price-Prediction.mlflow/#/experiments/0/runs/c2f6a022e4b44503bbd7ed4daa47bfed
🧪 View experiment at: https://dagshub.com/JavithNaseem-J/Laptop-Price-Prediction.mlflow/#/experiments/0
[2025-02-21 23:35:28,787 : ERROR : 2610076619 : Error in model evaluation: 'accuracy']
